In [1]:
import tensorflow as tf
from tensorflow import keras

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
housing = fetch_california_housing()

In [3]:
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [4]:
# モデル構築
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])

# コンパイル
model.compile(loss="mean_squared_error", optimizer="sgd")

history = model.fit(
    X_train, 
    y_train,
    epochs=20,
    validation_data=(X_valid, y_valid)
)

mse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3]
y_pred = model.predict(X_new)

Epoch 1/20
363/363 [==============================] - 1s 1ms/step - loss: 1.3971 - val_loss: 0.5603
Epoch 2/20
363/363 [==============================] - 0s 412us/step - loss: 0.6813 - val_loss: 0.5062
Epoch 3/20
363/363 [==============================] - 0s 411us/step - loss: 0.4739 - val_loss: 0.5105
Epoch 4/20
363/363 [==============================] - 0s 411us/step - loss: 0.4581 - val_loss: 0.4523
Epoch 5/20
363/363 [==============================] - 0s 409us/step - loss: 0.4294 - val_loss: 0.4361
Epoch 6/20
363/363 [==============================] - 0s 423us/step - loss: 0.4121 - val_loss: 0.4267
Epoch 7/20
363/363 [==============================] - 0s 409us/step - loss: 0.4189 - val_loss: 0.4195
Epoch 8/20
363/363 [==============================] - 0s 409us/step - loss: 0.4037 - val_loss: 0.4177
Epoch 9/20
363/363 [==============================] - 0s 408us/step - loss: 0.4119 - val_loss: 0.4087
Epoch 10/20
363/363 [==============================] - 0s 407us/step - loss: 0.3885 

# 関数型APIを使ったモデルの構築
### 非シーケンシャルニューラルネットワーク

- 入力の全てまたは一部を直接出力層に接続する。

In [5]:
input_ = keras.layers.Input(shape=X_train.shape[1:])   # 入力タイプを指定
hidden1 = keras.layers.Dense(30, activation="relu")(input_)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.Concatenate()([input_, hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_], outputs=[output])

In [6]:
# 2つのパスを連結
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.Concatenate()([input_A, hidden2])
output = keras.layers.Dense(1, name="output")(concat)
model = keras.Model(inputs=[input_A, input_B], outputs=[output])

In [8]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

history = model.fit(
    (X_train_A, X_train_B),
    y_train, 
    epochs=20,
    validation_data=((X_valid_A, X_valid_B), y_valid)
)

mse_test = model.evaluate((X_test_A, X_test_B), y_test)
y_pred = model.predict((X_new_A, X_new_B))

Epoch 1/20
363/363 [==============================] - 0s 604us/step - loss: 0.9108 - val_loss: 0.7313
Epoch 2/20
363/363 [==============================] - 0s 491us/step - loss: 0.7053 - val_loss: 0.6575
Epoch 3/20
363/363 [==============================] - 0s 486us/step - loss: 0.6451 - val_loss: 0.6223
Epoch 4/20
363/363 [==============================] - 0s 492us/step - loss: 0.6220 - val_loss: 0.6001
Epoch 5/20
363/363 [==============================] - 0s 551us/step - loss: 0.5921 - val_loss: 0.5828
Epoch 6/20
363/363 [==============================] - 0s 514us/step - loss: 0.5702 - val_loss: 0.5690
Epoch 7/20
363/363 [==============================] - 0s 492us/step - loss: 0.5482 - val_loss: 0.5574
Epoch 8/20
363/363 [==============================] - 0s 492us/step - loss: 0.5254 - val_loss: 0.5477
Epoch 9/20
363/363 [==============================] - 0s 491us/step - loss: 0.5256 - val_loss: 0.5395
Epoch 10/20
363/363 [==============================] - 0s 491us/step - loss: 0.529

In [9]:
# 補助出力の追加
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.Concatenate()([input_A, hidden2])
output = keras.layers.Dense(1, name="main_output")(concat)
aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)
model = keras.Model(inputs=[input_A, input_B], outputs=[output, aux_output])

In [10]:
model.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer="sgd")   # メイン出力の損失に大きな重みを与える。

history = model.fit(
    [X_train_A, X_train_B],
    [y_train, y_train], 
    epochs=20,
    validation_data=([X_valid_A, X_valid_B], [y_valid, y_valid])
)

total_loss, main_loss, aux_loss = model.evaluate([X_test_A, X_test_B], [y_test, y_test])
y_pred_main, y_pred_aux = model.predict((X_new_A, X_new_B))

Epoch 1/20
363/363 [==============================] - 1s 798us/step - loss: 1.6122 - main_output_loss: 1.4133 - aux_output_loss: 3.4026 - val_loss: 0.5898 - val_main_output_loss: 0.5279 - val_aux_output_loss: 1.1471
Epoch 2/20
363/363 [==============================] - 0s 564us/step - loss: 0.5584 - main_output_loss: 0.4970 - aux_output_loss: 1.1111 - val_loss: 0.5916 - val_main_output_loss: 0.5473 - val_aux_output_loss: 0.9903
Epoch 3/20
363/363 [==============================] - 0s 570us/step - loss: 0.5759 - main_output_loss: 0.5312 - aux_output_loss: 0.9782 - val_loss: 0.4974 - val_main_output_loss: 0.4589 - val_aux_output_loss: 0.8442
Epoch 4/20
363/363 [==============================] - 0s 632us/step - loss: 0.4735 - main_output_loss: 0.4346 - aux_output_loss: 0.8234 - val_loss: 0.4740 - val_main_output_loss: 0.4439 - val_aux_output_loss: 0.7456
Epoch 5/20
363/363 [==============================] - 0s 577us/step - loss: 0.4567 - main_output_loss: 0.4229 - aux_output_loss: 0.7615 

## サブクラス化APIを使用したダイナミックなモデルの構築
- ループを含んだり、形を変更したり、条件分岐したりといったダイナミックな動作を必要とするクラスを作成できる。

In [ ]:
class WideAndDeepModel